In [1]:
from selenium import webdriver
import selenium
import time
import pymongo
import requests
from bs4 import BeautifulSoup
import time
import json

import numpy as np
import pandas as pd

In [2]:
mc = pymongo.MongoClient()  # Connect to the MongoDB server using default settings

In [3]:

db = mc['movies_metadata']  # Use (or create) a database called 'movies_metadata'
test = db['test']  # Use (or create) a collection called 'test'


In [4]:
movies = db['movies'] #Create a collection called movies

In [5]:
mc.database_names()

['admin', 'local', 'movies_metadata', 'nyt_dump', 'tweets']

In [32]:
list(movies.collection.find())

NameError: name 'movies_metadata' is not defined

In [6]:
len(list(movies.find()))
#2017 movies populate movies.find first 510 records
#moving on...

1061

In [30]:
#2016 movies:
browser = webdriver.Firefox()
i = 519
browser.get("https://www.the-numbers.com/United-States/movies/year/2016")
while (browser.find_element_by_css_selector("#page_filling_chart > center:nth-child(7) > table:nth-child(3) > tbody:nth-child(1) > tr:nth-child(" + str(i) + ") > td:nth-child(5)").text != '$0'):
    
    #If the worldwide revenue is a nonzero amount... then go to the direct webpage for the movie
    #Then store that webpage's html code in the pymongo collection
    #The data on the webpage is sorted by worldwide box office revenue in descending order, with all the zero amounts 
    #at the bottom 
    url = browser.find_element_by_css_selector("#page_filling_chart > center:nth-child(7) > table:nth-child(3) > tbody:nth-child(1) > tr:nth-child(" + str(i) + ") > td:nth-child(1) > b:nth-child(1) > a:nth-child(1)").get_attribute('href')
    # pause between HTTP requests
    browser.get(url)
    DELAY = 5 + 5 * np.random.random() # time to wait between HTTP requests
    time.sleep(DELAY)
    html = browser.page_source
    movies.insert_one({'url': url,
                     'ts': time.time(),
                     'html': html
                     })
    print(i)
    i += 1
    browser.get("https://www.the-numbers.com/United-States/movies/year/2016")



519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552


In [18]:
#2015 movies:
browser = webdriver.Firefox()
i = 483
browser.get("https://www.the-numbers.com/United-States/movies/year/2015")
while (browser.find_element_by_css_selector("#page_filling_chart > center:nth-child(7) > table:nth-child(3) > tbody:nth-child(1) > tr:nth-child(" + str(i) + ") > td:nth-child(5)").text != '$0'):
    time.sleep(2)
    #If the worldwide revenue is a nonzero amount... then go to the direct webpage for the movie
    #Then store that webpage's html code in the pymongo collection
    url = browser.find_element_by_css_selector("#page_filling_chart > center:nth-child(7) > table:nth-child(3) > tbody:nth-child(1) > tr:nth-child(" + str(i) + ") > td:nth-child(1) > b:nth-child(1) > a:nth-child(1)").get_attribute('href')
    # pause between HTTP requests
    browser.get(url)
    DELAY = 5 + 5 * np.random.random() # time to wait between HTTP requests
    time.sleep(DELAY)
    html = browser.page_source
    movies.insert_one({'url': url,
                     'ts': time.time(),
                     'html': html
                     })
    print(i)
    i += 1
    browser.get("https://www.the-numbers.com/United-States/movies/year/2015")


483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523


In [ ]:
#page_filling_chart > center:nth-child(7) > table:nth-child(3) > tbody:nth-child(1) > tr:nth-child(523) > td:nth-child(5)

In [23]:
pd.read_html(list(movies.find())[-1]['html'])

[                              0        1        2
 0        Theatrical Performance      NaN      NaN
 1           Domestic Box Office  $83,992  Details
 2  Further financial details...      NaN      NaN,
                                                    0      1        2
 0                                             Record   Rank   Amount
 1  All Time Domestic Box Office (Rank 11,301-11,400)  11385  $83,992
 2  All Time Worldwide Box Office (Rank 15,301-15,...  15359  $83,992
 3  All Time Domestic Highest Grossing Limited Rel...   4217  $83,992,
                         0                                                  1
 0      Domestic Releases:  April 15th, 2016 (Limited) by Oscilloscope Pic...
 1          Video Release:            October 7th, 2016 by Oscilloscope Video
 2            MPAA Rating:                                          Not Rated
 3           Running Time:                                         57 minutes
 4            Comparisons:                  Create you

#What numbers do I need?
Title - from html code itself, should be located somewhere just before the first table

Items below can be extracted from pd.read_html, look into whether its easier to pull from this or directly from the unparsed mongodb format
Domestic Box office
Intl Box office 
Worldwide Box office

Release Date
MPAA Rating
Running Time
Source
Genre
Production Method
Creative Type
Production Companies


Lead Actor 1
Lead Actor 2
Lead Actor 3
Lead Actor 4

Director
Executive Producer
Producer

In [ ]:
#Figure out how to pull one number: Domestic Box Office
j = 0
while j != 99:
    my_tables = pd.read_html(list(test.find())[-1]['html'])
    for k in range(0,len(my_tables)):
        ind_table = my_tables[k]
        for m in range(0,len(ind_table)):
            if ind_table.iloc[m,0] == 'Domestic Box Office':
                dom = ind_table.iloc[m,1]
                print(dom)
    j = 99
        

In [99]:
len(list(movies.find()))

471

In [27]:
pd.read_html(list(test.find())[-2]['html'])

[                                 0               1        2
 0           Theatrical Performance             NaN      NaN
 1              Domestic Box Office    $936,662,225  Details
 2         International Box Office  $1,122,000,000  Details
 3             Worldwide Box Office  $2,058,662,225      NaN
 4          Home Market Performance             NaN      NaN
 5          Est. Domestic DVD Sales     $37,530,278  Details
 6      Est. Domestic Blu-ray Sales    $150,794,223  Details
 7  Total Est. Domestic Video Sales    $188,324,501      NaN
 8     Further financial details...             NaN      NaN,
                                    0     1               2
 0                             Record  Rank          Amount
 1       All Time Domestic Box Office     1    $936,662,225
 2  All Time International Box Office     4  $1,122,000,000
 3      All Time Worldwide Box Office     3  $2,058,662,225,
                           0                                                  1
 0      

In [41]:
list(test.find())[-2]['html'].index('<h1 itemprop="name">')

32767

In [55]:
list(test.find())[-1]['html'].index('</h1>')

34584

In [54]:
list(test.find())[-1]['html'][32787]

'>'

In [48]:
[list(test.find())[-2]['html'].index('<h1 itemprop="name">'),list(test.find())[-2]['html'].index('<h1 itemprop="name">') + 20]

[32767, 32787]

In [53]:
list(test.find())[-2]['html'][32787:32830]

'Star Wars Ep. VII: The Force Awakens (2015)'

In [82]:
movie_titles = []
# for i in range(len(list(test.find()))):
for i in range(10):
    title_start_index = list(movies.find())[i]['html'].index('<h1 itemprop="name">')
#     print(title_start_index)
#     print(title_start_index + 20)
    title_end_index = list(movies.find())[i]['html'].index('</h1>')
    print(list(movies.find())[i]['html'][(title_start_index + 20): title_end_index])

Star Wars Ep. VIII: The Last Jedi (2017)
Beauty and the Beast (2017)
The Fate of the Furious (2017)
Despicable Me 3 (2017)
Jumanji: Welcome to the Jungle (2017)
Spider-Man: Homecoming (2017)
Guardians of the Galaxy Vol 2 (2017)
Thor: Ragnarok (2017)
Wonder Woman (2017)
Coco (2017)


In [50]:
import sys
sys.getsizeof(list(movies.find()))

13904

In [83]:
url

'https://www.the-numbers.com/movie/Addicted-to-Fresno#tab=summary'

In [92]:
retrieved_document = movies.find_one({'url': 'https://www.the-numbers.com/movie/Star-Wars-Ep-VIII-The-Last-Jedi#tab=summary'})


In [89]:
import pymongo
import requests
from bs4 import BeautifulSoup
import time
import json

In [93]:
retrieved_document.keys()

dict_keys(['_id', 'url', 'ts', 'html'])

In [125]:
movies.find()[0]['url']

'https://www.the-numbers.com/movie/Star-Wars-Ep-VIII-The-Last-Jedi#tab=summary'

In [131]:
movie_titles = []
for i in range(15):
    retrieved_document = movies.find()[i]
    soup = BeautifulSoup(retrieved_document['html'], 'html.parser')
    movie_title = soup.select('h1')[0].text
    movie_titles.append(movie_title)
    print(movie_title)
    movie_titles
%timeit

Star Wars Ep. VIII: The Last Jedi (2017)
Beauty and the Beast (2017)
The Fate of the Furious (2017)
Despicable Me 3 (2017)
Jumanji: Welcome to the Jungle (2017)
Spider-Man: Homecoming (2017)
Guardians of the Galaxy Vol 2 (2017)
Thor: Ragnarok (2017)
Wonder Woman (2017)
Coco (2017)
Pirates of the Caribbean: Dead Men Tell No Tales (2017)
It (2017)
Justice League (2017)
Logan (2017)
Transformers: The Last Knight (2017)


In [77]:
movie_titles = []
for p in (list(test.find()))[0]['html']:
    try:
        title_element = p.find_element_by_css_selector('h1 itemprop="name"')
        title = title_element.text

#         price_whole_element = p.find_element_by_css_selector("span.sx-price-whole")
#         price = price_whole_element.text

#         price_fractional_element = p.find_element_by_css_selector(
#             "sup.sx-price-fractional")
#         price += ("." + price_fractional_element.text)
    except selenium.common.exceptions.NoSuchElementException:
        title_element = None
    movie_titles.append((title, price))

AttributeError: 'str' object has no attribute 'find_element_by_css_selector'

In [79]:
list(test.find())[0]['html']

'<html><head><script src="https://securepubads.g.doubleclick.net/gpt/pubads_impl_rendering_203.js"></script><script async="" type="text/javascript" src="https://www.googletagservices.com/tag/js/gpt.js"></script><script type="text/javascript" async="" src="https://apex.go.sonobi.com/trinity.js?key_maker={&quot;17c615db972ecad&quot;:&quot;d2fcf2d90d97f0c9c31f|300x250&quot;,&quot;18c31a9203f2324&quot;:&quot;0b8dadaf6d707611877c|300x250&quot;}&amp;cv=sbi_162b0179ced32f18&amp;ref=www.the-numbers.com"></script><script type="text/javascript" async="" src="https://prebid.districtm.ca/lib.js"></script><iframe id="21dbf1cbd3bcd6e" marginwidth="0" marginheight="0" style="border: 0px none; display: none;" scrolling="no" src="https://ads.pubmatic.com/AdServer/js/showad.js#PIX&amp;kdntuid=1&amp;p=95054" width="0" height="0" frameborder="0"></iframe><iframe id="2001e1a0cd511fe" marginwidth="0" marginheight="0" style="border: 0px none; display: none;" scrolling="no" src="about:blank" width="0" height=

In [35]:
import sys
sys.getsizeof(list(test.find()))



224

In [ ]:
metadata = pd.read_csv('data/movies_metadata.csv', low_memory= False)

In [ ]:
metadata.head()

In [5]:
len(metadata[metadata['revenue'] == 0])

38052

In [6]:
metadata[metadata['revenue'] == 0]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
11,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,12110,tt0112896,en,Dracula: Dead and Loving It,When a lawyer shows up at the vampire's doorst...,...,1995-12-22,0.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Dracula: Dead and Loving It,False,5.7,210.0
21,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,1710,tt0112722,en,Copycat,An agoraphobic psychologist and a female detec...,...,1995-10-27,0.0,124.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,One man is copying the most notorious killers ...,Copycat,False,6.5,199.0
23,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",NaN,12665,tt0114168,en,Powder,Harassed by classmates who won't accept his sh...,...,1995-10-27,0.0,111.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,An extraordinary encounter with another human ...,Powder,False,6.3,143.0
25,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,16420,tt0114057,en,Othello,The evil Iago pretends to be friend of Othello...,...,1995-12-15,0.0,123.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Envy, greed, jealousy and love.",Othello,False,7.0,33.0
27,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,17015,tt0114117,en,Persuasion,This film adaptation of Jane Austen's last nov...,...,1995-09-27,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Persuasion,False,7.4,36.0
29,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,37557,tt0115012,zh,摇啊摇，摇到外婆桥,A provincial boy related to a Shanghai crime f...,...,1995-04-30,0.0,108.0,"[{'iso_639_1': 'zh', 'name': '普通话'}]",Released,In 1930's Shanghai violence was not the proble...,Shanghai Triad,False,6.5,17.0
32,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 12, ...",NaN,78802,tt0114952,fr,"Guillaumet, les ailes du courage",NaN,...,1996-09-18,0.0,50.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Wings of Courage,False,6.8,4.0


In [9]:
(metadata[metadata['release_date'] > '2009-01-01'])

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
868,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,79782,tt1684935,en,Wenecja,An atmospheric coming-of-age story featuring a...,...,2010-05-25,0.0,110.0,"[{'iso_639_1': 'pl', 'name': 'Polski'}]",Released,NaN,Venice,False,7.5,4.0
1081,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,141210,tt2250194,en,The Sleepover,"The town of Derry has a secret, but no one tol...",...,2013-10-12,0.0,6.0,[],Released,NaN,The Sleepover,False,8.0,1.0
2114,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",http://www.thefarmerswifefilm.co.uk/,143750,tt2140519,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",...,2012-06-20,0.0,18.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Farmer's Wife,False,10.0,1.0
2564,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,tt1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,2012-03-22,0.0,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Nation. Underfed.,A Place at the Table,False,6.9,7.0
2778,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",NaN,171982,tt2378428,en,Romance,She's as hot as Britney Spears. Hotter. She pa...,...,2012-10-09,0.0,27.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Romance,False,0.0,0.0
4122,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",http://brokenkingdomfilm.com/#/kingdomcome,136558,tt2043893,en,Kingdom Come,"The documentary, Kingdom Come follows a first-...",...,2011-01-01,0.0,88.0,[],Released,The unmaking of independent film,Kingdom Come,False,8.0,1.0
4356,False,NaN,3512454,"[{'id': 18, 'name': 'Drama'}]",NaN,110428,tt2018086,fr,Camille Claudel 1915,"Winter, 1915. Confined by her family to an asy...",...,2013-03-13,115860.0,95.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Camille Claudel 1915,False,7.0,20.0
5766,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,78022,tt1790669,en,Da wu sheng,"In the 19th century, the Prince Regent of the ...",...,2011-09-08,0.0,95.0,"[{'iso_639_1': 'zh', 'name': '普通话'}, {'iso_639...",Released,NaN,My Kingdom,False,4.0,2.0
6563,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,66111,tt1351186,en,Luster,Thomas Luster is a troubled businessman who tr...,...,2010-12-12,0.0,0.0,[],Rumored,Know Your Enemy,Luster,False,4.8,4.0
11155,False,NaN,40000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",NaN,77221,tt1701210,en,Black Gold,"On the Arabian Peninsula in the 1930s, two war...",...,2011-12-21,5446000.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Black Gold,False,5.9,77.0


In [132]:
movies_2017_list = list(movies.find())

In [133]:
movies_2017_df = pd.DataFrame(movies_2017_list)

In [135]:
movies_2017_df.to_csv('data/movies_2017_html.csv')